<a href="https://colab.research.google.com/github/opread/Coursera-AI-Agents-do/blob/main/agent_creaza_functii_programmaticprompting1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!!pip install litellm

# Important!!!
#
# <---- Set your 'OPENAI_API_KEY' as a secret over there with the "key" icon
#
#
import os
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = api_key

In [2]:
from typing import List, Dict
from litellm import completion
import re
from pathlib import Path

def generate_response(messages: List[Dict]) -> str:
    """Call LLM to get response"""
    response = completion(
        model="openai/gpt-4",
        messages=messages,
        max_tokens=1024
    )
    # Defensive in case content is None
    return (response.choices[0].message.content or "").strip()

# Regex to capture first fenced code block, with or without "python"
_CODE_FENCE_RE = re.compile(r"```(?:\s*python)?\s*\n(.*?)\n```", re.DOTALL | re.IGNORECASE)

def extract_code_block(response: str) -> str:
    """Extract code block from response. If none, return original."""
    if "```" not in response:
        return response.strip()
    m = _CODE_FENCE_RE.search(response)
    if m:
        return m.group(1).strip()
    # fallback: split naive if regex fails to match closing fence format
    parts = response.split("```")
    if len(parts) >= 2:
        return parts[1].lstrip().removeprefix("python").lstrip()
    return response.strip()

def develop_custom_function():
    # Get user input for function description
    print("\nWhat kind of function would you like to create?")
    print("Example: 'A function that calculates the factorial of a number'")
    function_description = input("Your description: ").strip()

    # Initialize conversation with system prompt
    messages = [
        {"role": "system", "content": "You are a Python expert helping to develop a function."}
    ]

    # First prompt - Basic function
    messages.append({
        "role": "user",
        "content": (
            f"Write a Python function that {function_description}. "
            "Output the function in a ```python code block```."
        )
    })
    initial_function = generate_response(messages)

    # Parse the response to get the function code
    initial_function = extract_code_block(initial_function)

    print("\n=== Initial Function ===")
    print(initial_function)

    # Add assistant's response to conversation (code-only, no escaped backticks)
    messages.append({
        "role": "assistant",
        "content": f"```python\n{initial_function}\n```"
    })

    # Second prompt - Add documentation
    messages.append({
        "role": "user",
        "content": (
            "Add comprehensive documentation to this function, including description, parameters, "
            "return value, examples, and edge cases. Output the function in a ```python code block```."
        )
    })
    documented_function = generate_response(messages)
    documented_function = extract_code_block(documented_function)

    print("\n=== Documented Function ===")
    print(documented_function)

    # Add documentation response to conversation (code-only)
    messages.append({
        "role": "assistant",
        "content": f"```python\n{documented_function}\n```"
    })

    # Third prompt - Add test cases
    messages.append({
        "role": "user",
        "content": (
            "Add unittest test cases for this function, including tests for basic functionality, "
            "edge cases, error cases, and various input scenarios. Output the code in a ```python code block```."
        )
    })
    test_cases = generate_response(messages)
    test_cases = extract_code_block(test_cases)

    print("\n=== Test Cases ===")
    print(test_cases)

    # Generate filename from function description (safe and short)
    safe_stem = "".join(ch.lower() if ch.isalnum() or ch.isspace() else " " for ch in function_description)
    safe_stem = "_".join(safe_stem.split())[:30] or "function"
    filename = f"{safe_stem}.py"

    # Save final version
    Path(filename).write_text(documented_function + "\n\n" + test_cases + "\n", encoding="utf-8")

    return documented_function, test_cases, filename

if __name__ == "__main__":
    function_code, tests, filename = develop_custom_function()
    print(f"\nFinal code has been saved to {filename}")



What kind of function would you like to create?
Example: 'A function that calculates the factorial of a number'
Your description: calculates radical 

=== Initial Function ===
import math

def calculate_radical(number):
    try:
        result = math.sqrt(number)
        return result
    except ValueError:
        return "Error: You tried to calculate the square root of a negative number"

=== Documented Function ===
import math

def calculate_radical(number):
    """
    This function calculates the square root (also known as the radical) of a given number.

    Parameters:
    number (float or int): The number for which to calculate the square root. 

    Returns:
    float: The square root of the input number.
    str: If the input number is negative, the function returns an error message.

    Example:
    calculate_radical(4) --> output: 2.0
    calculate_radical(15) --> output: 3.872983346207417

    Edge Cases:
    calculate_radical(-1) --> output: "Error: You tried to calcula